In [70]:
import pandas as pd
import numpy as np
import os


def get_tfe(model_results):
    baseline_result  = model_results[model_results.eblc == 'baseline']
    model_results['TFE'] = (model_results.nrmse.values - baseline_result.nrmse.values)/baseline_result.nrmse.values
    baseline_result = baseline_result.append([baseline_result]*2, ignore_index=True)
    baseline_result.at[0, 'eblc'] = 'pmc'
    baseline_result.at[1, 'eblc'] = 'sz'
    baseline_result.at[2, 'eblc'] = 'swing'
    baseline_result['TFE'] = 0.
    return pd.concat([model_results, baseline_result])


def join_with_aus():
    aus_results = pd.DataFrame()
    for root, dirs, files in os.walk('../results/tfe/per_model/'):
        for result_file in files:
            if 'gru' in result_file or 'arima' in result_file:
                continue
            print(result_file)
            model_results = pd.read_csv(os.path.join(root, result_file))
            model_results = model_results[(model_results.data == 'aus')]
            model_results['model'] = result_file.split('_')[0]
            model_results = get_tfe(model_results)
            aus_results = pd.concat([aus_results, model_results])


    aus_results.reset_index(drop=True, inplace=True)
    aus_results.eblc = aus_results.eblc.str.upper()
    aus_results.model = aus_results.model.str.upper()
    aus_results.data = aus_results.data.str.upper()
    aus_results['tfe_metric'] = 'nrmse'
    aus_results['te_metric'] = 'nrmse'
    aus_results['te'] = aus_results['error']
    x = aus_results.eb.values
    aus_results.eb = np.where(x >= 1.0, x*0.01, x)
    aus_results = aus_results.rename({'nrmse': 'forecasting error', 'eblc': 'compression'}, axis=1)
    aus_results = aus_results.drop(['mae', 'rmse', 'rse', 'corr', 'data_corr'], axis=1)
    all_model_results = pd.read_csv('../results/tfe/all_models_results.csv')
    cr_df = pd.read_csv('../results/cr/all_cr.csv')
    cr_df = cr_df[(cr_df.data == 'aus')]
    cr_df.compression = cr_df.compression.str.upper()
    cr_df.data = cr_df.data.str.upper()
    cr_df.rename({'gzip': 'compression ratio', 'error_bound': 'eb'}, axis=1, inplace=True)
    cr_df.set_index(['data', 'compression', 'eb'], inplace=True)
    joined = cr_df.merge(aus_results, on=['data', 'compression', 'eb'])
    namr = pd.concat([all_model_results, joined]).reset_index(drop=True)
    namr.to_csv('../results/tfe/all_models_results.csv', index=False)
